In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import files

# Upload a file from your local system
uploaded = files.upload()

Saving reviews_dataset.csv to reviews_dataset.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

# Load your dataset
df = pd.read_csv('reviews_dataset.csv')
df = shuffle(df, random_state=42)

#encoding sentiment
label_encoder = LabelEncoder()
df['Sentiment_encoded'] = label_encoder.fit_transform(df['Sentiment'])

# Define features and target variable
X = df['ReviewContent']
y = df['Sentiment_encoded']

# Stratified split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b')

# Tokenize the training and testing data
max_length = 512
X_train_tokens = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt', max_length=max_length)
X_test_tokens = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt', max_length=max_length)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = ReviewDataset(X_train_tokens, y_train.values)
test_dataset = ReviewDataset(X_test_tokens, y_test.values)


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the model
model = AutoModelForSequenceClassification.from_pretrained('meta-llama/Llama-2-7b', num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)




Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

<ipython-input-56-d20167a9d392>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,1.237600
20,1.256700
30,1.222900
40,1.198500
50,1.129700
60,1.035200
70,0.979000
80,1.012600
90,0.960100
100,0.779600


TrainOutput(global_step=435, training_loss=0.832974568728743, metrics={'train_runtime': 623.982, 'train_samples_per_second': 11.14, 'train_steps_per_second': 0.697, 'total_flos': 1786052526804000.0, 'train_loss': 0.832974568728743, 'epoch': 3.0})

In [ ]:
# Evaluate the model
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

<ipython-input-56-d20167a9d392>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results: {'eval_loss': 0.7933262586593628, 'eval_runtime': 30.829, 'eval_samples_per_second': 32.21, 'eval_steps_per_second': 2.044, 'epoch': 3.0}


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

# Classification report
print("Classification Report (lemma3):")
print(classification_report(y_test, y_pred))

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (lemma3): {accuracy:.2f}")

<ipython-input-56-d20167a9d392>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Classification Report (RoBERTa):
              precision    recall  f1-score   support

           0       0.58      0.06      0.10       125
           1       0.00      0.00      0.00        53
           2       0.54      0.62      0.58       117
           3       0.77      0.94      0.85       698

    accuracy                           0.74       993
   macro avg       0.47      0.40      0.38       993
weighted avg       0.68      0.74      0.68       993

Accuracy (RoBERTa): 0.74


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_path = "/content/drive/MyDrive/RoBERTaM75best"

# Save the model
trainer.save_model(model_path)

In [ ]:
import pickle
import os




# Define file paths in Google Drive
drive_path = '/content/drive'

# File names for saving
train_file = '//content/drive/MyDrive/X_trainb.pkl'
test_file = '//content/drive/MyDrive/X_testb.pkl'
y_train_file = '//content/drive/MyDrive/Y_trainb.pkl'
y_test_file = '//content/drive/MyDrive/Y_testb.pkl'


# Save X_train
with open(os.path.join(drive_path, train_file), 'wb') as f:
    pickle.dump(X_train, f)

# Save X_test
with open(os.path.join(drive_path, test_file), 'wb') as f:
    pickle.dump(X_test, f)

# Save y_train
with open(os.path.join(drive_path, y_train_file), 'wb') as f:
    pickle.dump(y_train, f)

# Save y_test
with open(os.path.join(drive_path, y_test_file), 'wb') as f:
    pickle.dump(y_test, f)

print("Files saved to Google Drive successfully.")
